In [24]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '고양시 강송로 카페'  # 검색어


# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait


# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# 검색창 찾기
search = driver.find_element_by_css_selector('div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다

sleep(1)


# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    res
    soup


# 페이지 다운
def page_down(num):
    body = driver.find_element_by_css_selector('body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)


# frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(5)

# 매장 리스트
store_list = driver.find_elements_by_css_selector('._22p-O') #driver.find_elements_by_css_selector('._1EKsQ')
# 페이지 리스트
next_btn = driver.find_elements_by_css_selector('._2ky45 > a') #동일

# dictionary 생성
store_dict = {'매장정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:-1]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    print(len(next_btn))
    for data in range(len(store_list)):  # 매장 리스트 만큼
        page = driver.find_elements_by_css_selector('._3Apve') #driver.find_elements_by_css_selector('.OXiLu')
        print(data)
        print(len(store_list))
        page[data].click()
        sleep(2)
        try:
            # 상세 페이지로 이동
            switch_frame('entryIframe')
            time_wait(5, '._3XamX')
            # 스크롤을 맨밑으로 1초간격으로 내린다.
            for down in range(3):
                sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # -----매장명 가져오기-----
            store_name = driver.find_element_by_css_selector('._3XamX').text
            print(store_name)

            # -----카테고리 가져오기-----
            store_category = driver.find_element_by_css_selector('._3ocDE').text
            print(store_category)

            # -----주소(위치)-----
            try:
                store_addr_list = driver.find_elements_by_css_selector('._1aj6-')
                for i in store_addr_list:
                    store_addr = i.find_element_by_css_selector('._1h3B_').text
            except:
                pass
            print(store_addr)
            # -----전화번호 가져오기-----
            try:
                store_tel = driver.find_element_by_css_selector('._3ZA0S').text
            except:
                pass

            print(store_tel)
            # -----영업시간-----
            try:
                store_time_list = driver.find_elements_by_css_selector('._2vK84')  # 아니 태그가 그세 바뀌네ㅡ,.ㅡ
                for i in store_time_list:
                    store_time = i.find_element_by_css_selector('._3uEtO > time').text
            except:
                pass
            print(store_time)

            # -----썸네일 사진 주소-----
            try:
                thumb_list = driver.find_element_by_css_selector('.cb7hz') \
                    .value_of_css_property('background-image')  # css 속성명을 찾는다
                store_thumb = re.sub('url|"|\)|\(', '', thumb_list)  # url , (" ") 제거
            except:
                pass
            print(store_thumb)

            # ---- dict에 데이터 집어넣기----
            dict_temp = {
                'name': store_name,
                'category' : store_category,
                'tel': store_tel,
                'addr': store_addr,
                'time': store_time,
                'thumb': store_thumb
            }

            store_dict['매장정보'].append(dict_temp)
            store_name = None
            store_category = None
            store_tel = None
            store_addr = None
            store_time = None
            store_thumb = None

            print(f'{store_name} ...완료')
            switch_frame('searchIframe')
            sleep(1)

        except:
            print('ERROR!' * 3)

    # 다음 페이지 버튼
    if page[-1]:  # 마지막 매장일 경우 다음버튼 클릭
        next_btn[-1].click()
        sleep(2)
        store_list = driver.find_elements_by_css_selector('._22p-O')# 매장 리스트
    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을닫는다.

# json 파일로 저장
with open('data/store_data.json', 'w', encoding='utf-8') as f:
    json.dump(store_dict, f, indent=4, ensure_ascii=False)

C:\Users\admin\AppData\Local\Temp\ipykernel_25408\4169166428.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\4169166428.py:38: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  search = driver.find_element_by_css_selector('div.input_box > input.input_search')
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\4169166428.py:58: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body = driver.find_element_by_css_selector('body')
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\4169166428.py:70: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  store_list = driver.find_element

[크롤링 시작...]
4
0
50


C:\Users\admin\AppData\Local\Temp\ipykernel_25408\4169166428.py:99: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  store_name = driver.find_element_by_css_selector('._3XamX').text
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\4169166428.py:103: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  store_category = driver.find_element_by_css_selector('._3ocDE').text
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\4169166428.py:108: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  store_addr_list = driver.find_elements_by_css_selector('._1aj6-')
C:\Users\admin\AppData\Local\Temp\ipykernel_25408\4169166428.py:116: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, v

수원의아침 일산 벨라시타점
베이커리
경기 고양시 일산동구 강송로 33 벨라시타 본동 2층
3백석역 8번 출구에서278m
0507-1330-5766
None
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20210705_132%2F1625474343956J0ikO_PNG%2Fjj68eEZWBsR2_cEKLiYC61uA.png
None ...완료


C:\Users\admin\AppData\Local\Temp\ipykernel_25408\4169166428.py:132: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  thumb_list = driver.find_element_by_css_selector('.cb7hz') \


1
50
챔피언 벨라시타점
키즈카페,실내놀이터
경기 고양시 일산동구 강송로 33 벨라시타 지하1층 키즈카페 챔피언
3백석역 8번 출구에서278m
031-849-6627
11:00에 영업 시작
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220206_247%2F1644141339469zv1aJ_JPEG%2FKakaoTalk_20220206_185358549.jpg
None ...완료
2
50
오릴리 벨라시타점
카페,디저트
경기 고양시 일산동구 강송로 33 벨라시타 본동 2층
3백석역 8번 출구에서278m
0507-1330-5766
None
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20210705_132%2F1625474343956J0ikO_PNG%2Fjj68eEZWBsR2_cEKLiYC61uA.png
None ...완료
3
50
스타벅스 백석중앙로점
카페
경기 고양시 일산동구 강송로 33
3백석역 8번 출구에서278m
1522-3232
22:00에 영업 종료
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20190828_93%2F1566953601239OT9MQ_PNG%2FxX7Wv642gXMoTI0DAv0hRymS.png
None ...완료
4
50
잇츠브레드
베이커리
경기 고양시 일산동구 강송로 33 벨라시타 쇼핑몰 본동 1층 잇츠브레드
3백석역 8번 출구에서278m
0507-1323-5739
22:00에 영업 종료
https://search.pstatic.net/common/?autoRotate=t

C:\Users\admin\AppData\Local\Temp\ipykernel_25408\4169166428.py:168: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  store_list = driver.find_elements_by_css_selector('._22p-O')# 매장 리스트


4
0
7
리넥 본점 벨라시타
주방용품
경기 고양시 일산동구 강송로 33 벨라시타 지하 1층
3백석역 8번 출구에서278m
031-849-5754
None
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20210210_226%2F1612924071666GtjYF_JPEG%2FgStCvKafV2UJ8HNhhkFGVSQt.jpg
None ...완료
1
7
뿌염뿌염천연염색카페

경기 고양시 일산동구 강송로 115
3백석역 6번 출구에서919m
None
None
https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fapis.naver.com%2Fplace%2Fpanorama%2Fthumbnail%2F1445282418%2F0%3Fwidth%3D800%26height%3D400%26msgpad%3D1651834015274%26md%3DW%252FyEzS%252F317EpLShfhp8r7CR%252Fz9E%253D
None ...완료
2
7
바우바우
반려동물미용
경기 고양시 일산동구 강송로 33 벨라시타 본동 1층
3백석역 7번 출구에서582m
0507-1410-5772
22:00에 영업 종료
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220428_43%2F1651146741431Wnrst_JPEG%2F1650372399104.jpg
None ...완료
3
7
퀴즈노스 벨라시타점
샌드위치
경기 고양시 일산동구 강송로 33 지하1층 A-B 120호
3백석역 8번 출구에서278m
031-849-5791
22:00에 영업 종료
https://search.pstati

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '고양시 강선로 카페'  # 검색어


# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait


# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# 검색창 찾기
search = driver.find_element_by_css_selector('div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다

sleep(1)


# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    res
    soup


# 페이지 다운
def page_down(num):
    body = driver.find_element_by_css_selector('body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)


# frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(5)

# 매장 리스트
store_list = driver.find_elements_by_css_selector('._22p-O') #driver.find_elements_by_css_selector('._1EKsQ')
# 페이지 리스트
next_btn = driver.find_elements_by_css_selector('._2ky45 > a') #동일

# dictionary 생성
store_dict = {'매장정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    store_list
    for data in range(len(store_list)):  # 매장 리스트 만큼
        page = driver.find_elements_by_css_selector('._3Apve') #driver.find_elements_by_css_selector('.OXiLu')
        page[data].click()
        sleep(2)
        try:
            # 상세 페이지로 이동
            switch_frame('entryIframe')
            time_wait(5, '._3XamX')
            # 스크롤을 맨밑으로 1초간격으로 내린다.
            for down in range(3):
                sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # -----매장명 가져오기-----
            store_name = driver.find_element_by_css_selector('._3XamX').text
            print(store_name)

            # -----카테고리 가져오기-----
            store_category = driver.find_element_by_css_selector('._3ocDE').text
            print(store_category)

            # -----주소(위치)-----
            try:
                store_addr_list = driver.find_elements_by_css_selector('._1aj6-')
                for i in store_addr_list:
                    store_addr = i.find_element_by_css_selector('._1h3B_').text
            except:
                pass
            print(store_addr)
            # -----전화번호 가져오기-----
            try:
                store_tel = driver.find_element_by_css_selector('._3ZA0S').text
            except:
                pass

            print(store_tel)
            # -----영업시간-----
            try:
                store_time_list = driver.find_elements_by_css_selector('._2vK84')  # 아니 태그가 그세 바뀌네ㅡ,.ㅡ
                for i in store_time_list:
                    store_time = i.find_element_by_css_selector('._3uEtO > time').text
            except:
                pass
            print(store_time)

            html = driver.page_source 
            soup = BeautifulSoup(html, 'html.parser') 
            # print(soup) 
            notices = soup.select('바꿔') 

            resultlist = [] 

            for n in notices: 
                # print ('aria-hidden-', n['aria-hidden']) aria-expanded로 바꿔
                if n['aria-expanded']=='false': 
                    lis = n.select('li') 
                    for l in lis: 
                        result = dict() 
                        result['rank'] = l.select_one('.num_issue').text 
                        result['title']= l.select_one('.txt_issue').text 
                        result['url'] = l.a['href'] 
                        # print(l.select_one('.num_issue').text) 
                        # print(l.select_one('.txt_issue').text) 
                        # print('href=',l.a['href']) 
                        resultlist.append(result) 

            # -----썸네일 사진 주소-----
            try:
                thumb_list = driver.find_element_by_css_selector('.cb7hz') \
                    .value_of_css_property('background-image')  # css 속성명을 찾는다
                store_thumb = re.sub('url|"|\)|\(', '', thumb_list)  # url , (" ") 제거
            except:
                pass
            print(store_thumb)

            # ---- dict에 데이터 집어넣기----
            dict_temp = {
                'name': store_name,
                'category' : store_category,
                'tel': store_tel,
                'addr': store_addr,
                'time': store_time,
                'thumb': store_thumb
            }

            store_dict['매장정보'].append(dict_temp)
            store_name = None
            store_category = None
            store_tel = None
            store_addr = None
            store_time = None
            store_thumb = None

            print(f'{store_name} ...완료')
            switch_frame('searchIframe')
            sleep(1)

        except:
            print('ERROR!' * 3)

    # 다음 페이지 버튼
    if page[-1]:  # 마지막 매장일 경우 다음버튼 클릭
        next_btn[-1].click()
        sleep(2)
    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을닫는다.

# json 파일로 저장
with open('data/store_data.json', 'w', encoding='utf-8') as f:
    json.dump(store_dict, f, indent=4, ensure_ascii=False)